In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
from math import sqrt
from scipy.stats import norm, skew
from scipy.special import boxcox1p
    
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [3]:
df_train = pd.read_csv("../input/mydata/2017.csv")
# df_train2018 = pd.read_csv("/content/drive/MyDrive/oreeva/2018.csv")


In [4]:
df_test = pd.read_csv("../input/mydata/2019.csv")

In [5]:
print(f'df_train2017 {df_train.shape}')
# print(f'df_train2018 {df_train2018.shape}')
# print(f'df_test {df_test.shape}')

df_train2017 (992212, 43)


In [6]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992212 entries, 0 to 992211
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   transaction_id                  992212 non-null  object 
 1   created                         992212 non-null  object 
 2   duration                        992212 non-null  object 
 3   price                           992212 non-null  int64  
 4   old_new                         992212 non-null  object 
 5   ppd_category                    992212 non-null  object 
 6   property_type                   992212 non-null  object 
 7   flat                            95590 non-null   object 
 8   floor_level                     113197 non-null  object 
 9   number_of_rooms                 488846 non-null  float64
 10  total_floor_area                550761 non-null  float64
 11  epc_current_energy_rating       550761 non-null  object 
 12  epc_potential_en

In [7]:
df_train.describe()


,price,number_of_rooms,total_floor_area,epc_energy_consumption_current,epc_lighting_cost_current,epc_heating_cost_current,epc_hot_water_cost_current,epc_flat_storey_count,epc_number_open_fireplaces,epc_floor_height,latitude,longitude,closeststop,crimesin3km_antisocial,closestschool,closestpark,address_id,distance_center
count,9.922120e+05,488846.000000,550761.000000,550761.000000,550761.000000,550761.000000,550761.000000,46374.000000,490416.000000,308738.000000,981289.000000,981289.000000,970311.000000,981030.000000,929846.000000,907947.000000,9.922120e+05,981289.000000
mean,3.430209e+05,4.607099,91.285681,932.934124,80.054866,718.063476,142.841696,3.207465,0.200783,2.452155,52.340786,-1.353508,311.753490,2371.478555,485.204683,964.415709,9.772327e+07,6.210879
std,2.057961e+06,1.765542,54.153310,4616.730188,40.891643,552.759957,76.631545,2.540377,0.561604,0.640488,1.141627,1.329082,475.290107,2943.331264,432.445959,793.564470,4.938171e+06,7.306605
min,1.000000e+00,1.000000,0.000000,-1323.000000,-8.000000,-65.000000,-31.000000,0.000000,0.000000,0.000000,49.913207,-6.352647,0.532170,0.000000,0.000000,3.000000,8.857598e+07,0.003302
25%,1.400000e+05,3.000000,64.000000,196.000000,55.000000,401.000000,96.000000,2.000000,0.000000,2.330000,51.455900,-2.240896,96.812114,446.000000,243.000000,412.000000,9.336849e+07,1.791768
50%,2.248145e+05,4.000000,81.500000,262.000000,73.000000,606.000000,117.000000,3.000000,0.000000,2.400000,52.143151,-1.387420,161.098046,1430.000000,383.000000,712.000000,9.830963e+07,3.749439
75%,3.500000e+05,5.000000,105.300000,348.000000,96.000000,872.000000,164.000000,4.000000,0.000000,2.500000,53.364162,-0.258715,295.766537,3048.000000,576.000000,1241.000000,1.021145e+08,7.717438
max,7.075042e+08,90.000000,5327.890000,358454.000000,6760.000000,48391.000000,1395.000000,55.000000,50.000000,230.000000,55.797415,1.757928,4784.667983,23074.000000,4695.000000,4838.000000,1.056134e+08,83.744964


In [8]:
# df_train = pd.concat([df_train2017,df_train2018])
# len_train = len(df_train)
# len_train

In [9]:
# def check_df_concat(df1,df2,df_concat):
#   if (df1.shape[0] + df2.shape[0]) == df_concat.shape[0]:
#     return f"Correctly Joined. Shape : {df_concat.shape}" 
#   return "Error"

In [10]:
# check_df_concat(df_train2017,df_train2018,df_train)

</h3>Train Dataset</h3>

In [11]:
df_train.head()

,transaction_id,created,duration,price,old_new,ppd_category,property_type,flat,floor_level,number_of_rooms,...,building_number,latitude,longitude,postcode_sector,closeststop,crimesin3km_antisocial,closestschool,closestpark,address_id,distance_center
0,{5CF9903C-EF46-7F13-E053-6C04A8C074B1},2017-06-22,F,86787,Y,B,T,NaN,NaN,NaN,...,24,53.177751,-1.191828,NG19 8,130.419369,741.0,735.0,674.0,102877980,3.658300
1,{5F54B81D-F9BF-2B45-E053-6B04A8C01FB0},2017-10-13,F,805000,N,A,D,NaN,NaN,9.0,...,3,51.380891,-1.605473,SN8 3,446.805795,10.0,633.0,NaN,97804281,27.205352
2,{4C4EE000-1146-1854-E050-A8C063054F34},2017-03-08,F,275000,N,A,D,NaN,NaN,NaN,...,36,52.469943,-2.134401,DY5 3,310.743404,1576.0,845.0,745.0,101429204,5.659612
3,{5F54B81D-EAA3-2B45-E053-6B04A8C01FB0},2017-10-27,L,220000,N,A,T,NaN,NaN,NaN,...,63,52.377534,-1.762286,B93 8,265.388640,184.0,817.0,686.0,100150188,4.190032
4,{5CF9903C-2479-7F13-E053-6C04A8C074B1},2017-05-05,F,237000,N,A,D,NaN,NaN,NaN,...,1,52.531956,-1.376669,LE10 2,236.354546,643.0,582.0,761.0,93361793,14.267637


In [12]:
print("Len train set: ",len(df_train))
print("Shape of train set: ", df_train.shape)

Len train set:  992212
Shape of train set:  (992212, 43)


Test Dataset

In [13]:
df_test.describe()

,number_of_rooms,total_floor_area,epc_energy_consumption_current,epc_lighting_cost_current,epc_heating_cost_current,epc_hot_water_cost_current,epc_flat_storey_count,epc_number_open_fireplaces,epc_floor_height,latitude,longitude,closeststop,crimesin3km_antisocial,closestschool,closestpark,address_id,distance_center
count,25775.000000,28323.000000,28323.000000,28323.000000,28323.000000,28323.000000,2606.000000,25850.000000,15239.000000,28321.000000,28321.000000,28052.000000,28320.000000,26890.000000,26364.000000,2.832300e+04,28321.000000
mean,4.482211,89.842777,1059.175123,77.230160,672.075178,137.459270,2.988872,0.211141,2.450256,52.344054,-1.378285,297.752091,2250.969880,464.329900,948.511569,9.418641e+07,6.097563
std,1.580803,58.958723,4812.350197,59.975968,452.811068,71.059309,2.073938,0.557207,0.273533,1.145298,1.335869,454.904724,2665.934581,400.751073,773.612523,3.329168e+06,7.058303
min,1.000000,1.240000,-30.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,49.987482,-5.691800,2.286049,0.000000,0.000000,13.000000,8.857593e+07,0.028571
25%,3.000000,65.000000,197.000000,54.000000,395.000000,94.000000,2.000000,0.000000,2.330000,51.444818,-2.248804,97.357629,474.000000,240.000000,408.000000,9.122862e+07,1.777685
50%,4.000000,81.000000,262.000000,71.000000,583.000000,114.000000,3.000000,0.000000,2.400000,52.201565,-1.415831,157.941862,1428.000000,375.000000,706.000000,9.411824e+07,3.732491
75%,5.000000,102.000000,348.000000,92.000000,818.000000,156.000000,4.000000,0.000000,2.520000,53.372771,-0.295174,281.528629,2939.250000,558.000000,1217.000000,9.710424e+07,7.662969
max,40.000000,6886.100000,182162.000000,8253.000000,9140.000000,961.000000,28.000000,11.000000,20.000000,55.777929,1.754483,4757.317580,21896.000000,4593.000000,4798.000000,9.999101e+07,80.603386


In [14]:
df_test.shape

(28323, 42)

In [15]:
df_test.head()

,transaction_id,created,duration,old_new,ppd_category,property_type,flat,floor_level,number_of_rooms,total_floor_area,...,building_number,latitude,longitude,postcode_sector,closeststop,crimesin3km_antisocial,closestschool,closestpark,address_id,distance_center
0,{8355F009-9C32-55C5-E053-6B04A8C0D090},2019-01-28,F,N,A,D,NaN,NaN,6.0,94.00,...,9,52.373472,-2.303423,DY12 1,71.545593,946.0,137.0,686.0,91642184,4.047481
1,{8355F008-F8E5-55C5-E053-6B04A8C0D090},2019-02-15,F,N,A,S,NaN,NaN,4.0,66.00,...,59,53.562627,-2.198870,M24 6,89.657603,2307.0,496.0,1846.0,94054111,6.092467
2,{8355F009-83DD-55C5-E053-6B04A8C0D090},2019-02-13,L,N,A,F,NaN,Ground,3.0,59.00,...,3,54.935637,-1.595428,NE9 5,106.416768,4137.0,216.0,866.0,94836811,3.047846
3,{80E1AA98-9CB1-7BF8-E053-6C04A8C00BF2},2019-01-04,L,N,A,T,NaN,1st,2.0,38.00,...,104,50.957224,-2.611460,BA21 5,1402.637172,1008.0,267.0,514.0,89021226,2.317706
4,{80E1AA98-A8C4-7BF8-E053-6C04A8C00BF2},2019-01-09,F,N,A,T,NaN,NaN,5.0,93.72,...,12,53.539628,-1.149541,DN5 9,69.745660,4900.0,450.0,2123.0,91563029,2.038629


Correlation of features with target variable.

In [16]:
df_train.columns.sort_values()

Index(['address_id', 'building_name', 'building_number', 'city', 'closestpark',
       'closestschool', 'closeststop', 'created', 'crimesin3km_antisocial',
       'distance_center', 'duration', 'epc_built_form',
       'epc_current_energy_rating', 'epc_energy_consumption_current',
       'epc_flat_storey_count', 'epc_floor_height', 'epc_glazed_area',
       'epc_heating_cost_current', 'epc_hot_water_cost_current',
       'epc_lighting_cost_current', 'epc_number_open_fireplaces',
       'epc_potential_energy_rating', 'epc_property_type',
       'epc_roof_description', 'epc_roof_energy_eff', 'epc_walls_description',
       'epc_walls_energy_eff', 'epc_windows_description',
       'epc_windows_energy_eff', 'flat', 'floor_level', 'latitude',
       'longitude', 'number_of_rooms', 'old_new', 'postcode',
       'postcode_sector', 'ppd_category', 'price', 'property_type', 'street',
       'total_floor_area', 'transaction_id'],
      dtype='object')

In [17]:
# df_train.corr()
co = pd.DataFrame((df_train.corr())['price'].sort_values(ascending=False))
print(co)


                                   price
price                           1.000000
total_floor_area                0.103011
epc_lighting_cost_current       0.078283
crimesin3km_antisocial          0.069967
number_of_rooms                 0.067995
epc_heating_cost_current        0.066814
epc_flat_storey_count           0.059270
longitude                       0.039182
epc_number_open_fireplaces      0.037144
address_id                      0.025247
epc_floor_height                0.024482
epc_hot_water_cost_current      0.016348
closestschool                   0.013863
epc_energy_consumption_current  0.006752
closeststop                    -0.001172
closestpark                    -0.005499
distance_center                -0.015240
latitude                       -0.047006


Result shows that the total_floor_area, epc ratings, and as expected the rooms have strong relation with price. 
Negative values shows the inverse realtion with price. (Property which are near to city center are tends to be more expensive. 

In [18]:
# df_train.corr()
cor = (df_train.corr()**2)['price'].sort_values(ascending=False)
print(cor)

price                             1.000000
total_floor_area                  0.010611
epc_lighting_cost_current         0.006128
crimesin3km_antisocial            0.004895
number_of_rooms                   0.004623
epc_heating_cost_current          0.004464
epc_flat_storey_count             0.003513
latitude                          0.002210
longitude                         0.001535
epc_number_open_fireplaces        0.001380
address_id                        0.000637
epc_floor_height                  0.000599
epc_hot_water_cost_current        0.000267
distance_center                   0.000232
closestschool                     0.000192
epc_energy_consumption_current    0.000046
closestpark                       0.000030
closeststop                       0.000001
Name: price, dtype: float64


In [19]:
# cities = pd.DataFrame(df_train['city'].value_counts())
# print(len(cities))
# cities.head(5)

In [20]:
# df_train.total_floor_area.value_counts()

Latitude and Longitude are the loc of property.

#Missing Values 


In [24]:
# plt.figure(figsize=(20,15))
# sns.heatmap(df_train.isna(),yticklabels=False,cbar=False)


A very informative plot, here we can see which columns/features have null/nan values in training dataset. 

**White strips shows the presence of null/nan values.**


In [22]:
# df_train['epc_energy_consumption_current'].isna().sum()
#Filtering out the features which have nan values

# df_train_missing_data = pd.DataFrame(df_train.isna().sum().sort_values(ascending=False), columns=['#Missing_Values'])
# df_train_missing_data = df_train_missing_data[df_train_missing_data['#Missing_Values'] > 0]
# df_train_missing_data

In [25]:
# energy_fea=['epc_built_form',
#        'epc_current_energy_rating', 'epc_energy_consumption_current',
#        'epc_flat_storey_count', 'epc_floor_height', 'epc_glazed_area',
#        'epc_heating_cost_current', 'epc_hot_water_cost_current',
#        'epc_lighting_cost_current', 'epc_number_open_fireplaces',
#        'epc_potential_energy_rating', 'epc_property_type',
#        'epc_roof_description', 'epc_roof_energy_eff', 'epc_walls_description',
#        'epc_walls_energy_eff', 'epc_windows_description',
#        'epc_windows_energy_eff']

**Splitting Categorical and Numerical Features**

In [26]:
def get_cat_num_data(df):
    continuous = []
    categorical = []
    for i in df.columns:
        if df[i].dtype == "object":
            categorical.append(i)
        else: continuous.append(i)
    return continuous,categorical

In [27]:
data_cat_num =  get_cat_num_data(df_train)
continuous = data_cat_num[0]
categorical = data_cat_num[1]
print("Continuous:", continuous)
print("-")
print("Categorical:", categorical)

Continuous: ['price', 'number_of_rooms', 'total_floor_area', 'epc_energy_consumption_current', 'epc_lighting_cost_current', 'epc_heating_cost_current', 'epc_hot_water_cost_current', 'epc_flat_storey_count', 'epc_number_open_fireplaces', 'epc_floor_height', 'latitude', 'longitude', 'closeststop', 'crimesin3km_antisocial', 'closestschool', 'closestpark', 'address_id', 'distance_center']
-
Categorical: ['transaction_id', 'created', 'duration', 'old_new', 'ppd_category', 'property_type', 'flat', 'floor_level', 'epc_current_energy_rating', 'epc_potential_energy_rating', 'epc_property_type', 'epc_built_form', 'epc_glazed_area', 'epc_windows_description', 'epc_windows_energy_eff', 'epc_walls_description', 'epc_walls_energy_eff', 'epc_roof_description', 'epc_roof_energy_eff', 'postcode', 'city', 'street', 'building_name', 'building_number', 'postcode_sector']


In [28]:
def cate_nan_counts(df):
    isna_sum = df[categorical].isna().sum()
    NaN_columns = [i for i in isna_sum.index if df[i].isna().sum() !=0]
    df_NaN_values = df[NaN_columns]
    missing_categorical = pd.DataFrame(df_NaN_values.isna().sum(), columns = ["Missing values"])
    return missing_categorical.sort_values(["Missing values"], axis = 0, ascending = False)

####Categorical

In [29]:
cate_nan_counts(df_train)

,Missing values
flat,896622
floor_level,879015
building_name,839984
epc_roof_energy_eff,512212
epc_glazed_area,503365
epc_built_form,468103
epc_walls_energy_eff,448409
epc_windows_energy_eff,445710
epc_roof_description,441677
epc_walls_description,441483


In [30]:
print(df_train.street.mode()[0])
print(df_train.city.mode()[0])


HIGH STREET
BIRMINGHAM


In [31]:
df_m1_train = df_train.copy()

In [32]:
# p'flat',
# 'floor_level',
# 'building_name',
# 'epc_roof_energy_eff',
# 'epc_glazed_area',
# 'epc_built_form',
# 'epc_walls_energy_eff',
# 'epc_windows_energy_eff',
# 'epc_roof_description',
# 'epc_walls_description',
# 'epc_windows_description',
# 'epc_property_type',
# 'epc_potential_energy_rating',
# 'epc_current_energy_rating',
# 'building_number','city','street']

In [33]:
for i in ['flat','floor_level','building_name','epc_roof_energy_eff',
          'epc_glazed_area','epc_built_form','epc_walls_energy_eff',
          'epc_windows_energy_eff','epc_roof_description',
          'epc_walls_description','epc_windows_description','epc_property_type',
          'epc_potential_energy_rating','epc_current_energy_rating',
          'building_number',]:
          df_m1_train[i].fillna("NA", inplace = True)

In [34]:
for i in ['city','street']:
    df_m1_train[i].fillna(df_m1_train[i].mode()[0], inplace = True)

In [35]:
df_m1_train[categorical].isna().sum()

transaction_id                 0
created                        0
duration                       0
old_new                        0
ppd_category                   0
property_type                  0
flat                           0
floor_level                    0
epc_current_energy_rating      0
epc_potential_energy_rating    0
epc_property_type              0
epc_built_form                 0
epc_glazed_area                0
epc_windows_description        0
epc_windows_energy_eff         0
epc_walls_description          0
epc_walls_energy_eff           0
epc_roof_description           0
epc_roof_energy_eff            0
postcode                       0
city                           0
street                         0
building_name                  0
building_number                0
postcode_sector                0
dtype: int64

####Numerical

In [36]:
def num_nan_count(df):
    isna_sum = df[continuous].isna().sum()
    NaN_columns = [i for i in isna_sum.index if df[i].isna().sum() !=0]
    df_NaN_values = df[NaN_columns]
    missing_continuous = pd.DataFrame(df_NaN_values.isna().sum(), columns = ["Missing values"])
    return missing_continuous.sort_values(["Missing values"], axis = 0, ascending = False)



In [37]:
num_nan_count(df_train)

,Missing values
epc_flat_storey_count,945838
epc_floor_height,683474
number_of_rooms,503366
epc_number_open_fireplaces,501796
total_floor_area,441451
epc_energy_consumption_current,441451
epc_lighting_cost_current,441451
epc_heating_cost_current,441451
epc_hot_water_cost_current,441451
closestpark,84265


In [38]:
for i in ( ['epc_flat_storey_count', 
          'epc_number_open_fireplaces', 'epc_floor_height', 
          'latitude', 'longitude']):
  df_m1_train[i].fillna(0, inplace = True)


for j in  (['epc_energy_consumption_current', 'epc_lighting_cost_current',
  'epc_heating_cost_current','epc_hot_water_cost_current']):
  df_m1_train[j].fillna(df_m1_train[j].median(), inplace = True)

In [39]:
for i in (['number_of_rooms', 'total_floor_area','closeststop', 'crimesin3km_antisocial', 
          'closestschool', 'closestpark', 'distance_center']):
          df_m1_train[i].fillna(df_m1_train[i].median(),inplace = True)

In [40]:
df_m1_train[continuous].isna().sum()

price                             0
number_of_rooms                   0
total_floor_area                  0
epc_energy_consumption_current    0
epc_lighting_cost_current         0
epc_heating_cost_current          0
epc_hot_water_cost_current        0
epc_flat_storey_count             0
epc_number_open_fireplaces        0
epc_floor_height                  0
latitude                          0
longitude                         0
closeststop                       0
crimesin3km_antisocial            0
closestschool                     0
closestpark                       0
address_id                        0
distance_center                   0
dtype: int64

<h2>Analysis</h2>

In [41]:
df_m1_train.shape

(992212, 43)

In [42]:
# big_skew = [feature for feature in continuous if abs(df_m1_train[feature].skew()) > 0.75]
# big_skew

In [43]:
# for feature in big_skew:
#     df_m1_train[feature] = boxcox1p(df_m1_train[feature], 0.15)

# print("Skew:", round(df_m1_train['price'].skew(),2))

In [44]:
df_m1_train.columns

Index(['transaction_id', 'created', 'duration', 'price', 'old_new',
       'ppd_category', 'property_type', 'flat', 'floor_level',
       'number_of_rooms', 'total_floor_area', 'epc_current_energy_rating',
       'epc_potential_energy_rating', 'epc_property_type', 'epc_built_form',
       'epc_energy_consumption_current', 'epc_lighting_cost_current',
       'epc_heating_cost_current', 'epc_hot_water_cost_current',
       'epc_flat_storey_count', 'epc_glazed_area',
       'epc_number_open_fireplaces', 'epc_windows_description',
       'epc_windows_energy_eff', 'epc_walls_description',
       'epc_walls_energy_eff', 'epc_roof_description', 'epc_roof_energy_eff',
       'epc_floor_height', 'postcode', 'city', 'street', 'building_name',
       'building_number', 'latitude', 'longitude', 'postcode_sector',
       'closeststop', 'crimesin3km_antisocial', 'closestschool', 'closestpark',
       'address_id', 'distance_center'],
      dtype='object')

In [45]:
X_sub = df_test.copy()

In [46]:
X_sub.isna().sum()

transaction_id                        0
created                               0
duration                              0
old_new                               0
ppd_category                          0
property_type                         0
flat                              26143
floor_level                       22828
number_of_rooms                    2548
total_floor_area                      0
epc_current_energy_rating             0
epc_potential_energy_rating           0
epc_property_type                     0
epc_built_form                      632
epc_energy_consumption_current        0
epc_lighting_cost_current             0
epc_heating_cost_current              0
epc_hot_water_cost_current            0
epc_flat_storey_count             25717
epc_glazed_area                    2548
epc_number_open_fireplaces         2473
epc_windows_description               0
epc_windows_energy_eff              103
epc_walls_description                 3
epc_walls_energy_eff                259


In [47]:
# 'flat','floor_level','number_of_rooms', 'epc_built_form', 'epc_flat_storey_count','epc_glazed_area','epc_number_open_fireplaces','epc_windows_energy_eff',
# 'epc_walls_description','epc_walls_energy_eff','epc_roof_description','epc_roof_energy_eff','epc_floor_height ','building_name','building_number','latitude',
# 'longitude','closeststop','crimesin3km_antisocial','closestschool','closestpark','distance_center'  

In [48]:
data_test_cat_num = get_cat_num_data(X_sub)
continuous_test = data_test_cat_num[0]
categorical_test = data_test_cat_num[1]
print("Continuous:",continuous_test)
print("Categorical", categorical_test)

Continuous: ['number_of_rooms', 'total_floor_area', 'epc_energy_consumption_current', 'epc_lighting_cost_current', 'epc_heating_cost_current', 'epc_hot_water_cost_current', 'epc_flat_storey_count', 'epc_number_open_fireplaces', 'epc_floor_height', 'latitude', 'longitude', 'closeststop', 'crimesin3km_antisocial', 'closestschool', 'closestpark', 'address_id', 'distance_center']
Categorical ['transaction_id', 'created', 'duration', 'old_new', 'ppd_category', 'property_type', 'flat', 'floor_level', 'epc_current_energy_rating', 'epc_potential_energy_rating', 'epc_property_type', 'epc_built_form', 'epc_glazed_area', 'epc_windows_description', 'epc_windows_energy_eff', 'epc_walls_description', 'epc_walls_energy_eff', 'epc_roof_description', 'epc_roof_energy_eff', 'postcode', 'city', 'street', 'building_name', 'building_number', 'postcode_sector']


In [49]:
cate_nan_counts(X_sub)

,Missing values
flat,26143
building_name,24936
floor_level,22828
epc_roof_energy_eff,3279
epc_glazed_area,2548
building_number,1803
epc_built_form,632
epc_walls_energy_eff,259
epc_windows_energy_eff,103
epc_roof_description,19


In [50]:
for i in (['flat','building_name','floor_level','epc_roof_energy_eff','epc_glazed_area','building_number',
  'epc_built_form','epc_walls_energy_eff','epc_windows_energy_eff','epc_roof_description','epc_walls_description']):
    X_sub[i].fillna("NA", inplace = True)

In [51]:
X_sub[categorical_test].isna().sum()

transaction_id                 0
created                        0
duration                       0
old_new                        0
ppd_category                   0
property_type                  0
flat                           0
floor_level                    0
epc_current_energy_rating      0
epc_potential_energy_rating    0
epc_property_type              0
epc_built_form                 0
epc_glazed_area                0
epc_windows_description        0
epc_windows_energy_eff         0
epc_walls_description          0
epc_walls_energy_eff           0
epc_roof_description           0
epc_roof_energy_eff            0
postcode                       0
city                           0
street                         0
building_name                  0
building_number                0
postcode_sector                0
dtype: int64

In [52]:
for i in (['number_of_rooms','closeststop', 'crimesin3km_antisocial', 
          'closestschool', 'closestpark', 'distance_center']):
          X_sub[i].fillna(X_sub[i].median(),inplace = True)

In [53]:
for i in (['latitude','longitude']):
          X_sub[i].fillna(0,inplace = True)

In [54]:
for i in ( ['epc_flat_storey_count', 
          'epc_number_open_fireplaces', 'epc_floor_height']):
    X_sub[i].fillna(0, inplace = True)


In [55]:
X_sub.isna().sum()

transaction_id                    0
created                           0
duration                          0
old_new                           0
ppd_category                      0
property_type                     0
flat                              0
floor_level                       0
number_of_rooms                   0
total_floor_area                  0
epc_current_energy_rating         0
epc_potential_energy_rating       0
epc_property_type                 0
epc_built_form                    0
epc_energy_consumption_current    0
epc_lighting_cost_current         0
epc_heating_cost_current          0
epc_hot_water_cost_current        0
epc_flat_storey_count             0
epc_glazed_area                   0
epc_number_open_fireplaces        0
epc_windows_description           0
epc_windows_energy_eff            0
epc_walls_description             0
epc_walls_energy_eff              0
epc_roof_description              0
epc_roof_energy_eff               0
epc_floor_height            

In [56]:
#Converting Cate to Num 
df_train = pd.get_dummies(df_m1_train[:10000])
print(df_train.shape)

X_sub = pd.get_dummies(X_sub[:10000])
print(X_sub.shape)

(10000, 27382)
(10000, 35300)


In [57]:
X = df_train.drop(['price'],axis = 1)
y = df_train['price']

In [58]:
#Split Data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25, random_state=1234)

In [59]:
rf = RandomForestRegressor()
rf.fit(X_train,y_train)

RandomForestRegressor()

In [60]:
rf_pred = rf.predict(X_test)

In [85]:
max(rf_pred)

13255600.0

In [67]:
def mape(actual: np.ndarray, predicted: np.ndarray):
    return mean_absolute_percentage_error(actual, predicted)
# """
#     Mean Absolute Percentage Error
#     Properties:
#         + Easy to interpret
#         + Scale independent
#         - Biased, not symmetric
#         - Undefined when actual[t] == 0
#     Note: result is NOT multiplied by 100
#     """

In [86]:
xgb_model = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)


In [ ]:
xgb_model.fit(X_train,y_train)

[17:17:21] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [ ]:
xgb_pred = xgb_model.predict(X_test)

In [56]:
import lightgbm as lgb

In [57]:
modelsFinal = []

modelsFinal.append(('ENet', make_pipeline(RobustScaler(), ElasticNet())))
modelsFinal.append(('Lasso', make_pipeline(RobustScaler(), Lasso())))
modelsFinal.append(('RF',RandomForestRegressor()))
modelsFinal.append(('GB',GradientBoostingRegressor()))
modelsFinal.append(("KRR",KernelRidge()))
modelsFinal.append(("Xgb",xgb.XGBRegressor()))
modelsFinal.append(("LGB", lgb.LGBMRegressor()))


In [58]:
def compute_models(X, y):
    names = []
    scores = []
    frame = pd.DataFrame()
    
    for name, model in modelsFinal:
        model.fit(X, y)
        prediction = model.predict(X_submission)
        frame[name] = prediction
        scores.append(model.score(X_test,y_test))
        names.append(name)
    
    bla = pd.DataFrame(scores, index = names, columns = ["score"])
    print(bla)
    
    return frame